This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [ ]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np

In [105]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "./train1"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "./test1"    # e.g. './muffin-vs-chihuahua/test'

In [106]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
# DATA PREPROCESSING & AUGMENTATION (ResNet50 preprocessing)
# Using preprocess_input for ResNet50 (ImageNet mean subtraction & channel handling)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 1280 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 15 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [ ]:
# RESNET50 TRANSFER LEARNING MODEL ARCHITECTURE
# Load pretrained ResNet50 without top classifier
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
# Freeze base layers for initial training phase
base_model.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=base_model.input, outputs=output)

# Optional fine-tuning strategy (uncomment later):
# for layer in base_model.layers[-20:]:
#     layer.trainable = True

In [ ]:
# Compile with low LR suitable for frozen base transfer learning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 455ms/step - accuracy: 0.6195 - loss: 0.9513 - val_accuracy: 0.7000 - val_loss: 0.8061
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 455ms/step - accuracy: 0.6195 - loss: 0.9513 - val_accuracy: 0.7000 - val_loss: 0.8061
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 403ms/step - accuracy: 0.7227 - loss: 0.7509 - val_accuracy: 0.7281 - val_loss: 0.6804
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 403ms/step - accuracy: 0.7227 - loss: 0.7509 - val_accuracy: 0.7281 - val_loss: 0.6804
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 354ms/step - accuracy: 0.7531 - loss: 0.6509 - val_accuracy: 0.7812 - val_loss: 0.5981
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 354ms/step - accuracy: 0.7531 - loss: 0.6509 - val_accuracy: 0.7812 - val_loss: 0.5981
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 352ms/step - accuracy: 0.7930 - loss: 0.5684 - val_accuracy: 0.7812 - val_loss: 0.5793
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 352ms/step - accuracy: 0.7930 - loss: 0.5684 - val_accu

In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.8000 - loss: 0.4295
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.8000 - loss: 0.4295
Test Accuracy: 0.800000011920929
Test Accuracy: 0.800000011920929


In [ ]:
# SAVE THE MODEL
model.save('exercise_6_custom_capayan.h5')

In [ ]:
# SIMPLE INFERENCE SCRIPT (ResNet50-based model)
from tensorflow.keras.preprocessing import image

# Cache the loaded model to avoid re-loading on every prediction
_model_cache = {}

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    """Predict whether the image is Cat or Rabbit using the ResNet50 transfer model.
    Args:
        img_path (str): Path to the input image.
        model_path (str): Path to the saved Keras model (.h5).
    """
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}. Run training & save cells first.")
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image file not found: {img_path}. Check path and working directory.")

    # Load model once and reuse
    if model_path not in _model_cache:
        _model_cache[model_path] = tf.keras.models.load_model(model_path, compile=False)
    model = _model_cache[model_path]

    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)  # ResNet50 preprocessing
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model.predict(img_array)[0, 0])
    label = "Cat" if pred >= 0.5 else "Rabbit"
    confidence = pred if pred >= 0.5 else 1.0 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2f}) from raw score {pred:.2f}")


In [ ]:
# Example usage
predict_image("test1/cat/cat.208.jpg")
predict_image("test1/rabbit/images-2.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: Rabbit (confidence: 0.97) from raw score 0.03
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: Rabbit (confidence: 0.97) from raw score 0.03
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: Cat (confidence: 0.95) from raw score 0.95
Prediction: Cat (confidence: 0.95) from raw score 0.95
